In [1]:
%pip install roboflow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="L8vQ0Gye8h6aMz39BYDo")
project = rf.workspace("detection-gcxjj").project("text_detection_kh")
version = project.version(5)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


In [7]:
import os

def retain_only_class_zero(input_folder, output_folder):
    """
    Retains only the lines with class ID 0 in YOLO label files.
    
    Parameters:
        input_folder (str): Path to folder containing YOLO label files.
        output_folder (str): Path to save updated label files.
    """
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Loop through each file in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):  # Process only .txt files
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)
            
            with open(input_path, "r") as infile, open(output_path, "w") as outfile:
                for line in infile:
                    parts = line.strip().split()  # Split line into components
                    if len(parts) > 0 and parts[0] == '0':  # Keep only lines with class ID 0
                        outfile.write(line)  # Write the line unchanged to the output file
    
    print("Labels updated to retain only class ID 0 and saved to:", output_folder)

# Example usage
input_folder = os.path.join(dataset.location, "valid/labels")  # Use dataset location for input folder path
output_folder = os.path.join(dataset.location, "valid/labelss")  # Use dataset location for output folder path
retain_only_class_zero(input_folder, output_folder)


Labels updated to retain only class ID 0 and saved to: /Users/makchanna128gmail.com/Downloads/Test_yolov11_50/text_detection_kh-5/valid/labelss


In [9]:
def fix_invalid_class_ids(label_folder, valid_classes, replacement_class=0):
    for file_name in os.listdir(label_folder):
        if file_name.endswith(".txt"):
            file_path = os.path.join(label_folder, file_name)
            with open(file_path, "r") as f:
                lines = f.readlines()
            
            updated_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    class_id = int(parts[0])
                    if class_id not in valid_classes:
                        parts[0] = str(replacement_class)
                updated_lines.append(" ".join(parts) + "\n")
            
            with open(file_path, "w") as f:
                f.writelines(updated_lines)

# Example usage
input_folder = os.path.join(dataset.location, "valid/labels")  # Use dataset location for input folder path
valid_classes = [0]  # Only class ID 0 is valid
# Call the function to fix invalid class IDs
fix_invalid_class_ids(input_folder, valid_classes)


In [10]:
import os

def check_invalid_class_ids(label_folder, valid_classes):
    invalid_files = []
    for file_name in os.listdir(label_folder):
        if file_name.endswith(".txt"):
            with open(os.path.join(label_folder, file_name), "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) > 0 and int(parts[0]) not in valid_classes:
                        invalid_files.append(file_name)
                        break
    return invalid_files

label_folder = os.path.join(dataset.location, "valid/labels")
valid_classes = [0]  # Only class ID 0 is valid
invalid_files = check_invalid_class_ids(label_folder, valid_classes)

if invalid_files:
    print("Invalid class IDs found in the following label files:")
    print(invalid_files)
else:
    print("All label files are valid.")


All label files are valid.


In [11]:
%pip install label-studio

  Using cached label_studio-1.15.0-py3-none-any.whl.metadata (14 kB)
  Using cached Django-4.2.17-py3-none-any.whl.metadata (4.1 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached attr-0.3.1-py3-none-any.whl
  Using cached azure_storage_blob-12.24.0-py3-none-any.whl.metadata (26 kB)
  Using cached bleach-5.0.1-py3-none-any.whl.metadata (27 kB)
  Using cached boto-2.49.0-py2.py3-none-any.whl.metadata (7.3 kB)
  Using cached django_annoying-0.10.6-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached django_cors_headers-3.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached django_csp-3.7-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached django_debug_toolbar-3.2.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached django_environ-0.10.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached django_extensions-3.2.3-py3-none-any.whl.metadata (6.3 kB)
  Using cached django_filter-2.4.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached django_migration_lin

In [7]:
import yaml

path = "/teamspace/studios/this_studio/text_detection_kh-5/data.yaml"

with open(path, "r") as f:
    data_config = yaml.safe_load(f)

print("Classes:", data_config["names"])
print("Train Images Path:", data_config["train"])
print("Validation Images Path:", data_config["val"])
print("test Images Path:", data_config["test"])


Classes: {0: 'Yolov11kh'}
Train Images Path: ../train/images
Validation Images Path: ../valid/images
test Images Path: ../test/images


In [8]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [9]:

%pip install -U ultralytics

Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch

if torch.backends.mps.is_available():
    print("MPS is available!")
else:
    print("MPS is not available.")


MPS is not available.


In [11]:
import torch
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")


# Train the model
train_results = model.train(
  data=path,  # path to dataset YAML
  epochs=50,  # number of training epochs
  imgsz=640,  # training image size
  batch=16,  # batch size
  # Set the device to MPS if available
  single_cls = True # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

Ultralytics 8.3.54 🚀 Python-3.10.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA L40S, 45596MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/teamspace/studios/this_studio/text_detection_kh-5/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

train: Scanning /teamspace/studios/this_studio/text_detection_kh-5/train/labels... 4986 images, 0 backgrounds, 2067 corrupt: 100%|██████████| 4986/4986 [00:03<00:00, 1484.70it/s]

train: WARNING ⚠️ /teamspace/studios/this_studio/text_detection_kh-5/train/images/2024-10-31-10-30-37_jpg.rf.7b843325a2c49da2183051463723ac6c.jpg: ignoring corrupt image/label: Label class 894 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /teamspace/studios/this_studio/text_detection_kh-5/train/images/2024-10-31-10-30-37_jpg.rf.a15b9b6414bb64c3f22268c1bd31cda9.jpg: ignoring corrupt image/label: Label class 894 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /teamspace/studios/this_studio/text_detection_kh-5/train/images/2024-10-31-10-30-37_jpg.rf.d0214bf03f90efcffef7d5bed5e735f6.jpg: ignoring corrupt image/label: Label class 894 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /teamspace/studios/this_studio/text_detection_kh-5/train/images/2024-10-31-10-32-51_jpg.rf.29cc9022da0015c8d0b5b4ae84ac713e.jpg: ignoring corrupt image/label: Label class 894 exceeds dataset class count 1. Possible class

train: New cache created: /teamspace/studios/this_studio/text_detection_kh-5/train/labels.cache


val: Scanning /teamspace/studios/this_studio/text_detection_kh-5/valid/labels... 445 images, 0 backgrounds, 0 corrupt: 100%|██████████| 445/445 [00:00<00:00, 1309.13it/s]

...

      50/50      2.48G     0.9491     0.5614      1.056         12        640: 100%|██████████| 183/183 [00:13<00:00, 13.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:01<00:00, 11.71it/s]

                   all        445       1548      0.845      0.549      0.649       0.42



50 epochs completed in 0.217 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.54 🚀 Python-3.10.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA L40S, 45596MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:01<00:00,  7.30it/s]


                   all        445       1548      0.845      0.549      0.649       0.42
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train


In [16]:
from ultralytics import YOLO

# Load the trained model
model_path = "../weights/best.pt"
model = YOLO(model_path)

# Path to your dataset YAML file
data_path = path

# Validate the model
results = model.val(data=data_path)

# Extract and print the evaluation metrics
metrics = results.box
print("Evaluation Metrics:")
print(f"mAP@0.5: {metrics.map50.item():.4f}")
print(f"mAP@0.5:0.95: {metrics.map.item():.4f}")
print(f"Precision: {metrics.p.item():.4f}")  # Use 'p' for precision
print(f"Recall: {metrics.r.item():.4f}")    # Use 'r' for recall
print(f"F1-Score: {2 * (metrics.p.item() * metrics.r.item()) / (metrics.p.item() + metrics.r.item()):.4f}")  # F1 is computed
print(f"Inference Time per Image (ms): {results.speed['inference']:.2f}")

Ultralytics 8.3.54 🚀 Python-3.10.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA L40S, 45596MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /teamspace/studios/this_studio/text_detection_kh-5/valid/labels.cache... 445 images, 0 backgrounds, 0 corrupt: 100%|██████████| 445/445 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1544, len(boxes) = 1548. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:02<00:00, 10.69it/s]


                   all        445       1548      0.845      0.549      0.649      0.422
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val
Evaluation Metrics:
mAP@0.5: 0.6493
mAP@0.5:0.95: 0.4217
Precision: 0.8449
Recall: 0.5489
F1-Score: 0.6655
Inference Time per Image (ms): 1.16
